# Eager
TensorFlow 的 Eager Execution 是一种命令式编程环境，可立即评估操作，无需构建图：操作会返回具体的值，而不是构建以后再运行的计算图。这样能让您轻松地开始使用 TensorFlow 和调试模型，并且还减少了样板代码。要遵循本指南，请在交互式 python 解释器中运行下面的代码示例。

Eager Execution 是一个灵活的机器学习平台，用于研究和实验，可提供：

- 直观的界面 - 自然地组织代码结构并使用 Python 数据结构。快速迭代小模型和小型数据集。
- 更轻松的调试功能 - 直接调用操作以检查正在运行的模型并测试更改。使用标准 Python 调试工具进行即时错误报告。
- 自然控制流程 - 使用 Python 控制流程而不是图控制流程，简化了动态模型的规范。

Eager Execution 支持大多数 TensorFlow 操作和 GPU 加速。有关在 Eager Execution 中运行的示例集合，请参阅：tensorflow/contrib/eager/python/examples。

In [2]:
import tensorflow as tf

tf.executing_eagerly()

True

In [3]:
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))

hello, [[4.]]


启用 Eager Execution 会改变 TensorFlow 操作的行为方式 - 现在它们会立即评估并将值返回给 Python。tf.Tensor 对象会引用具体值，而不是指向计算图中的节点的符号句柄。由于不需要构建稍后在会话中运行的计算图，因此使用 print() 或调试程序很容易检查结果。评估、输出和检查张量值不会中断计算梯度的流程。

Eager Execution 适合与 NumPy 一起使用。NumPy 操作接受 tf.Tensor 参数。TensorFlow 数学运算将 Python 对象和 NumPy 数组转换为 tf.Tensor 对象。tf.Tensor.numpy 方法返回对象的值作为 NumPy ndarray。

In [4]:
a = tf.constant([[1, 2],
                 [3, 4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [5]:
# Broadcasting support
b = tf.add(a, 1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [6]:
print(a * b)

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [7]:
import numpy as np

c = np.multiply(a, b)
print(c)

[[ 2  6]
 [12 20]]


In [8]:
print(a.numpy())

[[1 2]
 [3 4]]


## 1.动态控制流
Eager Execution 的一个主要好处是，在执行模型时，主机语言的所有功能都可用。因此，编写 fizzbuzz 很容易（举例而言）：

In [10]:
def fizzbuzz(max_num):
    counter = tf.constant(0)
    max_num = tf.convert_to_tensor(max_num)
    for num in range(1, max_num.numpy()+1):
        num = tf.constant(num)
        if int(num % 3) == 0 and int(num % 5) == 0:
            print('FizzBuzz')
        elif int(num % 3) == 0:
            print('Fizz')
        elif int(num % 5) == 0:
            print('Buzz')
        else:
            print(num.numpy())
        counter += 1

In [11]:
fizzbuzz(15)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz


## 2.构建模型
许多机器学习模型通过组合层来表示。将 TensorFlow 与 Eager Execution 结合使用时，您可以编写自己的层或使用在 tf.keras.layers 程序包中提供的层。

虽然您可以使用任何 Python 对象表示层，但 TensorFlow 提供了便利的基类 tf.keras.layers.Layer。您可以通过继承它实现自己的层：

In [12]:
class MySimpleLayer(tf.keras.layers.Layer):
    def __init__(self, output_units):
        super(MySimpleLayer, self).__init__()
        self.output_units = output_units

    def build(self, input_shape):
        #生成方法在第一次使用层时被调用。
        #在Buffd（）上创建变量允许您的形状
        #依赖于输入形状，从而消除了用户指定
        #完整形状的需要。如果您已经知道变量的完整形状，则可以在初始化期间创建变量。
        self.kernel = self.add_variable(
          "kernel", [input_shape[-1], self.output_units])

    def call(self, input):
        return tf.matmul(input, self.kernel)

请使用 tf.keras.layers.Dense 层（而不是上面的 MySimpleLayer），因为它具有其功能的超集（它也可以添加偏差）。将层组合成模型时，可以使用 tf.keras.Sequential 表示由层线性堆叠的模型。它非常适合用于基本模型：

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, input_shape=(784,)),  # must declare input shape
  tf.keras.layers.Dense(10)
])

或者，通过继承 tf.keras.Model 将模型整理为类。这是一个本身也是层的层容器，允许 tf.keras.Model 对象包含其他 tf.keras.Model 对象。

## 3.Eager 训练
### 3.1计算梯度
自动微分对于实现机器学习算法（例如用于训练神经网络的反向传播）来说很有用。在 Eager Execution 期间，请使用 tf.GradientTape 跟踪操作以便稍后计算梯度。

tf.GradientTape 是一种选择性功能，可在不跟踪时提供最佳性能。由于在每次调用期间都可能发生不同的操作，因此所有前向传播操作都会记录到“磁带”中。要计算梯度，请反向播放磁带，然后放弃。特定的 tf.GradientTape 只能计算一个梯度；随后的调用会抛出运行时错误。

In [13]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
    loss = w * w

grad = tape.gradient(loss, w)
print(grad)  # => tf.Tensor([[ 2.]], shape=(1, 1), dtype=float32)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


### 3.2训练模型
以下示例将创建一个多层模型，该模型会对标准 MNIST 手写数字进行分类。它演示了在 Eager Execution 环境中构建可训练图的优化器和层 API。

In [18]:
# Fetch and format the mnist data
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices(
  (tf.cast(mnist_images[...,tf.newaxis]/255, tf.float32),
   tf.cast(mnist_labels,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

# Build the model
mnist_model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16,[3,3], activation='relu'),
  tf.keras.layers.Conv2D(16,[3,3], activation='relu'),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10)
])

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

loss_history = []



for (batch, (images, labels)) in enumerate(dataset.take(400)):
    if batch % 80 == 0:
        print()
    print('.', end='')
    with tf.GradientTape() as tape:
        logits = mnist_model(images, training=True)
        loss_value = loss_object(labels, logits)

    loss_history.append(loss_value.numpy())
    grads = tape.gradient(loss_value, mnist_model.variables)
    optimizer.apply_gradients(zip(grads, mnist_model.variables))


................................................................................
................................................................................
................................................................................
................................................................................
................................................................................

### 3.3变量和优化器
tf.Variable 对象会存储在训练期间访问的可变 tf.Tensor 值，以更加轻松地实现自动微分。模型的参数可以作为变量封装在类中。

通过将 tf.Variable 与 tf.GradientTape 结合使用可以更好地封装模型参数。例如，上面的自动微分示例可以重写为：

In [19]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.W = tf.Variable(5., name='weight')
        self.B = tf.Variable(10., name='bias')
    def call(self, inputs):
        return inputs * self.W + self.B

# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 2000
training_inputs = tf.random.normal([NUM_EXAMPLES])
noise = tf.random.normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# The loss function to be optimized
def loss(model, inputs, targets):
    error = model(inputs) - targets
    return tf.reduce_mean(tf.square(error))

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, [model.W, model.B])

# Define:
# 1. A model.
# 2. Derivatives of a loss function with respect to model parameters.
# 3. A strategy for updating the variables based on the derivatives.
model = MyModel()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

print("Initial loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))

# Training loop
for i in range(300):
    grads = grad(model, training_inputs, training_outputs)
    optimizer.apply_gradients(zip(grads, [model.W, model.B]))
    if i % 20 == 0:
        print("Loss at step {:03d}: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

print("Final loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))

Initial loss: 70.607
Loss at step 000: 67.796
Loss at step 020: 30.317
Loss at step 040: 13.898
Loss at step 060: 6.697
Loss at step 080: 3.536
Loss at step 100: 2.146
Loss at step 120: 1.535
Loss at step 140: 1.265
Loss at step 160: 1.147
Loss at step 180: 1.094
Loss at step 200: 1.071
Loss at step 220: 1.061
Loss at step 240: 1.056
Loss at step 260: 1.054
Loss at step 280: 1.053
Final loss: 1.053
W = 2.9813506603240967, B = 2.00752854347229


### 3.4对象

In [20]:
# 变量即对象
if tf.test.is_gpu_available():
    with tf.device("gpu:0"):
        v = tf.Variable(tf.random.normal([1000, 1000]))
        v = None  # v no longer takes up GPU memory

In [21]:
# 对象保存
x = tf.Variable(6.0)
checkpoint = tf.train.Checkpoint(x=x)

x.assign(1.0)
checkpoint.save('./ckpt/')

x.assign(8.0)
checkpoint.restore(tf.train.latest_checkpoint('./ckpt/'))
print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>


In [ ]:
# 模型保持
import os
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16,[3,3], activation='relu'),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10)
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
checkpoint_dir = './ck_model_dir'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
root = tf.train.Checkpoint(optimizer=optimizer,
                           model=model)

root.save(checkpoint_prefix)
root.restore(tf.train.latest_checkpoint(checkpoint_dir))

### 3.5自动微分高级内容
动态模型
tf.GradientTape 也可用于动态模型。这个回溯线搜索算法示例看起来像普通的 NumPy 代码，除了存在梯度并且可微分，尽管控制流比较复杂：

In [22]:
def line_search_step(fn, init_x, rate=1.0):
    with tf.GradientTape() as tape:
    # Variables are automatically recorded, but manually watch a tensor
        tape.watch(init_x)
        value = fn(init_x)
    grad = tape.gradient(value, init_x)
    grad_norm = tf.reduce_sum(grad * grad)
    init_value = value
    while value > init_value - rate * grad_norm:
        x = init_x - rate * grad
        value = fn(x)
        rate /= 2.0
    return x, value

计算梯度的其他函数
tf.GradientTape 是用于计算梯度的强大接口，还有另一种 Autograd 样式的 API 可用于自动微分。如果只用张量和梯度函数编写数学代码，而不使用 tf.Variables，则这些函数非常有用：

- tf.gradients_function - 返回一个函数，该函数会计算其输入函数参数相对于其参数的导数。输入函数参数必须返回一个标量值。当返回的函数被调用时，它会返回一个 tf.Tensor 对象列表：输入函数的每个参数各对应一个元素。因为任何相关信息都必须作为函数参数传递，所以如果依赖于许多可训练参数，则会变得很难处理。
- tf.value_and_gradients_function - 与 tfe.gradients_function 相似，但是当返回的函数被调用时，除了输入函数相对于其参数的导数列表之外，它还会返回输入函数的值。


In [25]:
# 自定义梯度
@tf.custom_gradient
def clip_gradient_by_norm(x, norm):
    y = tf.identity(x)
    def grad_fn(dresult):
        return [tf.clip_by_norm(dresult, norm), None]
    return y, grad_fn

In [26]:
# 自定义梯度可以提供数值稳定的梯度
def log1pexp(x):
    return tf.math.log(1 + tf.exp(x))

def grad_log1pexp(x):
    with tf.GradientTape() as tape:
        tape.watch(x)
        value = log1pexp(x)
    return tape.gradient(value, x)
print(grad_log1pexp(tf.constant(0.)).numpy())
# However, x = 100 fails because of numerical instability.
print(grad_log1pexp(tf.constant(100.)).numpy())

0.5
nan


In [30]:
## 8.使用gpu提升性能
import time

def measure(x, steps):
    # TensorFlow initializes a GPU the first time it's used, exclude from timing.
    tf.matmul(x, x)
    start = time.time()
    for i in range(steps):
        x = tf.matmul(x, x)
  # tf.matmul can return before completing the matrix multiplication
  # (e.g., can return after enqueing the operation on a CUDA stream).
  # The x.numpy() call below will ensure that all enqueued operations
  # have completed (and will also copy the result to host memory,
  # so we're including a little more than just the matmul operation
  # time).
    _ = x.numpy()
    end = time.time()
    return end - start

shape = (1000, 1000)
steps = 200
print("Time to multiply a {} matrix by itself {} times:".format(shape, steps))

# Run on CPU:
with tf.device("/cpu:0"):
    print("CPU: {} secs".format(measure(tf.random.normal(shape), steps)))

# Run on GPU, if available:
    if tf.test.is_gpu_available():
        with tf.device("/gpu:0"):
            print("GPU: {} secs".format(measure(tf.random.normal(shape), steps)))
    else:
        print("GPU: not found")

Time to multiply a (1000, 1000) matrix by itself 200 times:
CPU: 6.608656883239746 secs
GPU: not found
